In [18]:
import requests
import pandas as pd
from collections import defaultdict

# ─── CONFIG ─────────────────────────────────────────────────────────────────────
think_tank_ids = [3544410, 5508333, 333333]   # Torsten, Max, Phil
youngsters_ids = [1584965, 2767628, 666666]    # Tommi, Pat, Frej

names = {
    3544410: "Torsten",
    5508333:  "Max",
    333333:  "Phil",
    1584965:  "Tommi",
    2767628:  "Pat",
    666666:  "Frej",
}

# Helper: name -> id
name_to_id = {v: k for k, v in names.items()}

# ─── FINAL SCHEDULE (exactly as provided) ───────────────────────────────────────
# Each GW maps to three pairings as ("Youngster", "Think Tank") by name
final_schedule_by_name = {
    1:  [("Frej","Phil"), ("Tommi","Max"),   ("Pat","Torsten")],
    2:  [("Frej","Max"),  ("Tommi","Torsten"), ("Pat","Phil")],
    3:  [("Frej","Torsten"), ("Tommi","Phil"), ("Pat","Max")],
    4:  [("Frej","Phil"), ("Tommi","Max"),   ("Pat","Torsten")],
    5:  [("Frej","Max"),  ("Tommi","Torsten"), ("Pat","Phil")],
    6:  [("Frej","Torsten"), ("Tommi","Phil"), ("Pat","Max")],
    7:  [("Frej","Phil"), ("Tommi","Max"),   ("Pat","Torsten")],
    8:  [("Frej","Max"),  ("Tommi","Torsten"), ("Pat","Phil")],
    9:  [("Frej","Torsten"), ("Tommi","Phil"), ("Pat","Max")],
    10: [("Frej","Phil"), ("Tommi","Max"),   ("Pat","Torsten")],
    11: [("Frej","Max"),  ("Tommi","Torsten"), ("Pat","Phil")],
    12: [("Frej","Torsten"), ("Tommi","Phil"), ("Pat","Max")],
    13: [("Frej","Phil"), ("Tommi","Max"),   ("Pat","Torsten")],
    14: [("Frej","Max"),  ("Tommi","Torsten"), ("Pat","Phil")],
    15: [("Frej","Torsten"), ("Tommi","Phil"), ("Pat","Max")],
    16: [("Frej","Phil"), ("Tommi","Max"),   ("Pat","Torsten")],
    17: [("Frej","Max"),  ("Tommi","Torsten"), ("Pat","Phil")],
    18: [("Frej","Torsten"), ("Tommi","Phil"), ("Pat","Max")],
}

# Convert schedule names -> ids
schedule = defaultdict(list)  # {gw: [(id1, id2), ...]}
for gw, pairs in final_schedule_by_name.items():
    for p1_name, p2_name in pairs:
        p1 = name_to_id[p1_name]
        p2 = name_to_id[p2_name]
        schedule[gw].append((p1, p2))

n_gws = max(schedule.keys())
group_ids = think_tank_ids + youngsters_ids
# ────────────────────────────────────────────────────────────────────────────────

# 1) Fetch each manager’s per‑GW FPL points
points = {tid: {} for tid in group_ids}
for tid in group_ids:
    url = f"https://fantasy.premierleague.com/api/entry/{tid}/history/"
    resp = requests.get(url); resp.raise_for_status()
    for ev in resp.json().get("current", []):
        points[tid][ev["event"]] = ev["points"]

# 2) ONE ROW PER PLAYER PER WEEK (your requested first dataframe)
rows_weeks = []
for gw in range(1, n_gws + 1):
    for pid in group_ids:
        rows_weeks.append({
            "gameweek":    gw,
            "player_id":   pid,
            "player_name": names[pid],
            "fpl_points":  points[pid].get(gw, 0)
        })
df_details = pd.DataFrame(rows_weeks)

# 3) Compute match wins from the fixed schedule (ties = 0 for both)
match_rows = []
for gw, pairs in schedule.items():
    for a_id, b_id in pairs:
        a_pts = points[a_id].get(gw, 0)
        b_pts = points[b_id].get(gw, 0)
        if a_pts > b_pts:
            match_rows += [{"gameweek": gw, "player_id": a_id, "match_point": 1},
                           {"gameweek": gw, "player_id": b_id, "match_point": 0}]
        elif b_pts > a_pts:
            match_rows += [{"gameweek": gw, "player_id": a_id, "match_point": 0},
                           {"gameweek": gw, "player_id": b_id, "match_point": 1}]
        else:
            match_rows += [{"gameweek": gw, "player_id": a_id, "match_point": 0},
                           {"gameweek": gw, "player_id": b_id, "match_point": 0}]
df_matchpoints = pd.DataFrame(match_rows)

# 4) Manager summary: total FPL points + total wins
totals_points = (
    df_details.groupby(["player_id","player_name"], as_index=False)["fpl_points"]
    .sum()
    .rename(columns={"fpl_points": "total_fpl_points"})
)
totals_wins = (
    df_matchpoints.groupby("player_id", as_index=False)["match_point"]
    .sum()
    .rename(columns={"match_point": "total_wins"})
)
df_scores = (
    totals_points.merge(totals_wins, on="player_id", how="left")
    .fillna({"total_wins": 0})
    .astype({"total_wins": int})
)

# 6) Build overall Think Tank vs Youngsters scoreboard

# First, figure out for each player which team they're in
player_team = {}
for pid in think_tank_ids:
    player_team[pid] = "Think Tank"
for pid in youngsters_ids:
    player_team[pid] = "Youngsters"

# Aggregate match points per team per GW
df_team_gw = (
    df_matchpoints
    .assign(team=lambda df: df["player_id"].map(player_team))
    .groupby(["gameweek", "team"], as_index=False)["match_point"]
    .sum()
    .rename(columns={"match_point": "team_gw_points"})
)

# Now total across all GWs for final scoreboard
df_team_scoreboard = (
    df_team_gw
    .groupby("team", as_index=False)["team_gw_points"]
    .sum()
    .rename(columns={"team_gw_points": "total_match_points"})
    .sort_values("total_match_points", ascending=False)
)









In [16]:
df_matchpoints

,gameweek,player_id,match_point
0,1,666666,0
1,1,333333,0
2,1,1584965,0
3,1,5508333,0
4,1,2767628,0
...,...,...,...
103,18,3544410,0
104,18,1584965,0
105,18,333333,0
106,18,2767628,0


In [12]:
totals_wins

,player_id,total_wins
0,333333,0
1,666666,0
2,1584965,0
3,2767628,0
4,3544410,0
5,5508333,0


In [14]:
totals_points

,player_id,player_name,total_fpl_points
0,333333,Phil,0
1,666666,Frej,0
2,1584965,Tommi,0
3,2767628,Pat,0
4,3544410,Torsten,0
5,5508333,Max,0


In [15]:
df_scores

,player_id,player_name,total_fpl_points,total_wins
0,333333,Phil,0,0
1,666666,Frej,0,0
2,1584965,Tommi,0,0
3,2767628,Pat,0,0
4,3544410,Torsten,0,0
5,5508333,Max,0,0


In [19]:
df_team_scoreboard

,team,total_match_points
0,Think Tank,0
1,Youngsters,0


In [20]:
df_team_gw

,gameweek,team,team_gw_points
0,1,Think Tank,0
1,1,Youngsters,0
2,2,Think Tank,0
3,2,Youngsters,0
4,3,Think Tank,0
5,3,Youngsters,0
6,4,Think Tank,0
7,4,Youngsters,0
8,5,Think Tank,0
9,5,Youngsters,0


In [25]:
# streamlit_app.py
import requests
import pandas as pd
import streamlit as st
from collections import defaultdict

# ──────────────────────────────────────────────────────────────────────────────
# Page / Layout
# ──────────────────────────────────────────────────────────────────────────────
st.set_page_config(
    page_title="FPL: Think Tank vs Youngsters",
    page_icon="⚽",
    layout="wide"
)

st.title("⚽ FPL: Think Tank vs Youngsters — Head‑to‑Head Live")

# ──────────────────────────────────────────────────────────────────────────────
# Config: IDs, Names, Groups, Schedule
# ──────────────────────────────────────────────────────────────────────────────
THINK_TANK = [3544410, 222222, 333333]  # Torsten, Max, Phil
YOUNGSTERS = [444444, 555555, 666666]   # Tommi, Pat, Frej

NAMES = {
    3544410: "Torsten",
    222222:  "Max",
    333333:  "Phil",
    444444:  "Tommi",
    555555:  "Pat",
    666666:  "Frej",
}
NAME_TO_ID = {v: k for k, v in NAMES.items()}
PLAYER_TO_TEAM = {**{pid: "Think Tank" for pid in THINK_TANK},
                  **{pid: "Youngsters" for pid in YOUNGSTERS}}
ALL_IDS = THINK_TANK + YOUNGSTERS

# Exact schedule (Youngster vs Think Tank by names) — GW1..GW18
FINAL_SCHEDULE_BY_NAME = {
    1:  [("Frej","Phil"), ("Tommi","Max"),   ("Pat","Torsten")],
    2:  [("Frej","Max"),  ("Tommi","Torsten"), ("Pat","Phil")],
    3:  [("Frej","Torsten"), ("Tommi","Phil"), ("Pat","Max")],
    4:  [("Frej","Phil"), ("Tommi","Max"),   ("Pat","Torsten")],
    5:  [("Frej","Max"),  ("Tommi","Torsten"), ("Pat","Phil")],
    6:  [("Frej","Torsten"), ("Tommi","Phil"), ("Pat","Max")],
    7:  [("Frej","Phil"), ("Tommi","Max"),   ("Pat","Torsten")],
    8:  [("Frej","Max"),  ("Tommi","Torsten"), ("Pat","Phil")],
    9:  [("Frej","Torsten"), ("Tommi","Phil"), ("Pat","Max")],
    10: [("Frej","Phil"), ("Tommi","Max"),   ("Pat","Torsten")],
    11: [("Frej","Max"),  ("Tommi","Torsten"), ("Pat","Phil")],
    12: [("Frej","Torsten"), ("Tommi","Phil"), ("Pat","Max")],
    13: [("Frej","Phil"), ("Tommi","Max"),   ("Pat","Torsten")],
    14: [("Frej","Max"),  ("Tommi","Torsten"), ("Pat","Phil")],
    15: [("Frej","Torsten"), ("Tommi","Phil"), ("Pat","Max")],
    16: [("Frej","Phil"), ("Tommi","Max"),   ("Pat","Torsten")],
    17: [("Frej","Max"),  ("Tommi","Torsten"), ("Pat","Phil")],
    18: [("Frej","Torsten"), ("Tommi","Phil"), ("Pat","Max")],
}
N_GWS = max(FINAL_SCHEDULE_BY_NAME.keys())

# Convert schedule to IDs: {gw: [(idA, idB), ...]}
SCHEDULE = defaultdict(list)
for gw, pairs in FINAL_SCHEDULE_BY_NAME.items():
    for a_name, b_name in pairs:
        SCHEDULE[gw].append((NAME_TO_ID[a_name], NAME_TO_ID[b_name]))

# ──────────────────────────────────────────────────────────────────────────────
# Caching & Data Fetch
# ──────────────────────────────────────────────────────────────────────────────
# show something immediately so the page isn't blank
st.caption("App loaded — preparing data…")

@st.cache_data(ttl=15 * 60, show_spinner=False)
def fetch_player_points(player_id: int) -> dict:
    url = f"https://fantasy.premierleague.com/api/entry/{player_id}/history/"
    try:
        r = requests.get(url, timeout=6)  # shorter timeout
        r.raise_for_status()
        pts = {int(ev["event"]): int(ev["points"]) for ev in r.json().get("current", [])}
        return pts
    except Exception as e:
        # fail fast with empty data
        return {}

@st.cache_data(ttl=15 * 60, show_spinner=False)
def build_data_with_status():
    ALL_IDS = list(THINK_TANK) + list(YOUNGSTERS)
    points = {}

    # visual status block so the page isn't blank
    with st.status("Fetching FPL points…", expanded=True) as status:
        for i, pid in enumerate(ALL_IDS, start=1):
            st.write(f"Fetching {NAMES[pid]} ({pid})")
            pts = fetch_player_points(pid)
            if not pts:
                st.warning(f"No data for {NAMES[pid]} — using 0s (network or API issue).")
            points[pid] = pts
        status.update(label="Fetch complete", state="complete")

    # (the rest is identical to before, building df_player_weekly, df_fixtures, df_player_summary,
    #  df_team_weekly, df_team_scoreboard from the 'points' dict)
    # --- BEGIN original build logic ---
    rows_weeks = []
    for gw in range(1, N_GWS + 1):
        for pid in ALL_IDS:
            rows_weeks.append({
                "gameweek":    gw,
                "player_id":   pid,
                "player_name": NAMES[pid],
                "team":        PLAYER_TO_TEAM[pid],
                "fpl_points":  points[pid].get(gw, 0),
            })
    df_player_weekly = pd.DataFrame(rows_weeks)

    match_rows = []
    for gw, pairs in SCHEDULE.items():
        for a_id, b_id in pairs:
            a_pts = points[a_id].get(gw, 0)
            b_pts = points[b_id].get(gw, 0)
            if a_pts > b_pts:
                a_mp, b_mp, winner = 1, 0, NAMES[a_id]
            elif b_pts > a_pts:
                a_mp, b_mp, winner = 0, 1, NAMES[b_id]
            else:
                a_mp, b_mp, winner = 0, 0, "Draw"
            match_rows.append({
                "gameweek": gw,
                "home_id": a_id, "home_name": NAMES[a_id], "home_team": PLAYER_TO_TEAM[a_id],
                "home_score": a_pts, "home_match_point": a_mp,
                "away_id": b_id, "away_name": NAMES[b_id], "away_team": PLAYER_TO_TEAM[b_id],
                "away_score": b_pts, "away_match_point": b_mp,
                "winner": winner,
            })
    df_fixtures = pd.DataFrame(match_rows).sort_values(["gameweek", "home_name"])

    player_wins = []
    for pid in THINK_TANK + YOUNGSTERS:
        wins = 0
        for _, r in df_fixtures.iterrows():
            if r["home_id"] == pid:
                wins += r["home_match_point"]
            elif r["away_id"] == pid:
                wins += r["away_match_point"]
        player_wins.append({
            "player_id": pid, "player_name": NAMES[pid],
            "team": PLAYER_TO_TEAM[pid], "wins": wins,
            "total_fpl_points": int(df_player_weekly.loc[df_player_weekly.player_id == pid, "fpl_points"].sum())
        })
    df_player_summary = pd.DataFrame(player_wins).sort_values(["wins","total_fpl_points"], ascending=[False, False])

    team_weekly_rows = []
    for gw in range(1, N_GWS + 1):
        # Total FPL points for the team in this GW
        tt_fpl = int(df_player_weekly.query("gameweek==@gw and team=='Think Tank'")["fpl_points"].sum())
        yo_fpl = int(df_player_weekly.query("gameweek==@gw and team=='Youngsters'")["fpl_points"].sum())
        tt_mp = 0
        yo_mp = 0
        for _, r in df_fixtures[df_fixtures.gameweek == gw].iterrows():
            tt_mp += (r["home_match_point"] if r["home_team"] == "Think Tank" else r["away_match_point"])
            yo_mp += (r["home_match_point"] if r["home_team"] == "Youngsters" else r["away_match_point"])
        team_weekly_rows += [
            {"gameweek": gw, "team": "Think Tank", "team_fpl_points": tt_fpl, "team_match_points": tt_mp},
            {"gameweek": gw, "team": "Youngsters", "team_fpl_points": yo_fpl, "team_match_points": yo_mp},
        ]
    df_team_weekly = pd.DataFrame(team_weekly_rows)

    df_team_scoreboard = (
        df_team_weekly.groupby("team", as_index=False)["team_match_points"]
        .sum()
        .rename(columns={"team_match_points": "total_match_points"})
        .sort_values("total_match_points", ascending=False)
    )

    return df_player_weekly, df_fixtures, df_player_summary, df_team_weekly, df_team_scoreboard

2025-08-12 19:14:56.499 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-12 19:14:56.500 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-12 19:14:56.501 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-12 19:14:56.501 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-12 19:14:56.502 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-12 19:14:56.502 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-12 19:14:56.503 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-12 19:14:56.503 No runtime found, using MemoryCacheStorageManager
2025-08-12 19:14:56.505 No runtime found, us